In [1]:
from distributed import Client
client = Client()
client

/Users/scott/anaconda3/lib/python3.6/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:58951 Dashboard: http://127.0.0.1:58952/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [2]:
# import noisy_mnist
# chunk_size = 70_000 // 3
# _X, _y = noisy_mnist.dataset()
# X = _X[:chunk_size * 3]
# y = _y[:chunk_size * 3]

In [4]:
import dask.array as da
# n, d = _X.shape
# X = da.from_array(_X, chunks=(n // 3, d))
# y = da.from_array(_y, chunks=n // 3)
# X, y

In [5]:
from autoencoder import Autoencoder, NegLossScore
import torch
# from sklearn.model_selection import ParameterSampler
import torch

def trim_params(**kwargs):
    if kwargs['optimizer'] != 'Adam':
        kwargs.pop('optimizer__amsgrad', None)
    if kwargs['optimizer'] == 'Adam':
        kwargs.pop('optimizer__lr', None)
    if kwargs['optimizer'] != 'SGD':
        kwargs.pop('optimizer__nesterov', None)
        kwargs.pop('optimizer__momentum', None)
    kwargs['optimizer'] = getattr(torch.optim, kwargs['optimizer'])
    return kwargs

class TrimParams(NegLossScore):
    def set_params(self, **kwargs):
        kwargs = trim_params(**kwargs)
        return super().set_params(**kwargs)

model = TrimParams(
    module=Autoencoder,
    criterion=torch.nn.BCELoss,
    warm_start=True,
    train_split=None,
    max_epochs=1,
    callbacks=[]
)

In [6]:
import numpy as np

params = {
    'module__init': ['xavier_uniform_',
                     'xavier_normal_',
                     'kaiming_uniform_',
                     'kaiming_normal_',
                    ],
    'module__activation': ['ReLU', 'LeakyReLU', 'ELU', 'PReLU'],
    'optimizer': ["SGD"] * 5 + ["Adam"] * 2,
    'batch_size': [32, 64, 128, 256, 512],
    'optimizer__lr': np.logspace(1, -1.5, num=1000),
    'optimizer__weight_decay': [0]*200 + np.logspace(-5, -3, num=1000).tolist(),
    'optimizer__nesterov': [True],
    'optimizer__momentum': np.linspace(0, 1, num=1000),
    'train_split': [None],
}

In [16]:
from sklearn.model_selection import ParameterSampler
param = list(ParameterSampler(params, 4, random_state=3))[0]

model.set_params(**param)

<class '__main__.TrimParams'>[uninitialized](
  module=<class 'autoencoder.Autoencoder'>,
  module_=Autoencoder(
    (encoder): Sequential(
      (0): Linear(in_features=784, out_features=784, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace)
      (2): Linear(in_features=784, out_features=196, bias=True)
      (3): LeakyReLU(negative_slope=0.01, inplace)
    )
    (decoder): Sequential(
      (0): Linear(in_features=196, out_features=784, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace)
      (2): Linear(in_features=784, out_features=784, bias=True)
      (3): Sigmoid()
    )
  ),
  module__activation=LeakyReLU,
  module__init=kaiming_uniform_,
)